# Zadanie 1
Wybierz jeden z plików csv z poprzednich ćwiczeń (Mini Kurs, Actors.csv, Names.csv ect..) lub inny plik z dbfs:/databricks-datasets/ i na jego podstawie stwórz schemat danych. Przykład na wykładzie. 

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType

#schemat dla high_temps.csv
schema = StructType([
    StructField("date", StringType(), False),
    StructField("temp", FloatType(), False)
])



Użyj funckji spark.read. i stwórz DataFrame (DataFrameReader) wczytując plik z użyciem schematu, który stworzyłeś. 

In [0]:
#wczytanie danych wedlug schematu
high_temps_df = spark.read.csv("dbfs:/databricks-datasets/weather/high_temps", schema=schema, header=True)

display(high_temps_df.limit(10))
high_temps_df.printSchema()


date,temp
2015-01-01,42.0
2015-01-02,42.0
2015-01-03,41.0
2015-01-04,51.0
2015-01-05,54.0
2015-01-06,54.0
2015-01-07,46.0
2015-01-08,46.0
2015-01-09,50.0
2015-01-10,46.0


root
 |-- date: string (nullable = true)
 |-- temp: float (nullable = true)



# Zadanie 2

## Użycie Read Modes.  

Wykorzystaj posiadane pliki bądź użyj nowe.  Użyj Sparka do odczytania jednego pliku i użyj wszystkich typów read modes. Poprawny plik nie wywoła żadnych efektów, więc popsuj dane tak aby każda z read modes zadziałał. 

In [0]:
#popsucie - temp jako string
schema_error = StructType([
    StructField("date", FloatType(), True),
    StructField("temp", FloatType(), True) 
])


In [0]:

#PERMISSIVE błędne wartości na NULL
df_permissive = spark.read.format("csv").option("header", "true").option("mode", "PERMISSIVE") \
    .schema(schema_error).load("dbfs:/databricks-datasets/weather/high_temps")
display(df_permissive.limit(10))


date,temp
null,42.0
null,42.0
null,41.0
null,51.0
null,54.0
null,54.0
null,46.0
null,46.0
null,50.0
null,46.0


In [0]:
#DROPMALFORMED usunięcie błędnych wierszy
df_dropmalformed = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED") \
    .schema(schema_error).load("dbfs:/databricks-datasets/weather/high_temps")
display(df_dropmalformed.limit(10))

date,temp


In [0]:
#FAILFAST natychmiastowy błąd
df_failfast = spark.read.format("csv").option("header", "true").option("mode", "FAILFAST") \
    .schema(schema_error).load("dbfs:/databricks-datasets/weather/high_temps")
display(df_failfast.limit(10))

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 13) (ip-10-172-193-136.us-west-2.compute.internal executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/databricks-datasets/weather/high_temps.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:704)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:673)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:796)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:496)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:486)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:82)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:208)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:179)
	at org.apache.spark.scheduler.Task.$anonfun$run$5(Task.scala:142)
	at com.databricks.unity.EmptyHandle$.runWithAndClose(UCSHandle.scala:126)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:142)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:97)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:904)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1741)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:907)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:761)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
Caused by: org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1936)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:103)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:507)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apache.spark.util.CompletionIterator.hasNext(CompletionIterator.scala:31)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:619)
	... 29 more
Caused by: org.apache.spark.sql.catalyst.util.BadRecordException: java.lang.NumberFormatException: For input string: "2015-01-01"
Caused by: 

# Zadanie 3 

## Użycie DataFrameWriter. 

Zapisz jeden z wybranych plików do formatów (‘.parquet’, ‘.json’). Sprawdź, czy dane są zapisane poprawnie, użyj do tego spark.read (DataFrameReader).  

In [0]:
#zapisanie danych w Parquet
high_temps_df.write.mode("overwrite").parquet("dbfs:/tmp/high_temps.parquet")

#zapisanie danych w JSON
high_temps_df.write.mode("overwrite").json("dbfs:/tmp/high_temps.json")

#odczytanie danych z Parquet
df_parquet = spark.read.parquet("dbfs:/tmp/high_temps.parquet")
display(df_parquet.limit(10))

#odczytanie danych z JSON
df_json = spark.read.json("dbfs:/tmp/high_temps.json")
display(df_json.limit(10))


date,temp
2015-01-01,42.0
2015-01-02,42.0
2015-01-03,41.0
2015-01-04,51.0
2015-01-05,54.0
2015-01-06,54.0
2015-01-07,46.0
2015-01-08,46.0
2015-01-09,50.0
2015-01-10,46.0


date,temp
2015-01-01,42.0
2015-01-02,42.0
2015-01-03,41.0
2015-01-04,51.0
2015-01-05,54.0
2015-01-06,54.0
2015-01-07,46.0
2015-01-08,46.0
2015-01-09,50.0
2015-01-10,46.0
